## Demo notebook to check out the Google Places API and the results

In [1]:
import requests
from openai import OpenAI
import os
import nltk
import sqlite3

# Connect to database

In [2]:
conn = sqlite3.connect('reviews.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS review_summaries (
    id TEXT PRIMARY KEY,
    review_summary TEXT,
    date DATETIME DEFAULT CURRENT_TIMESTAMP
)''')

In [3]:
cursor.execute('''CREATE TABLE IF NOT EXISTS review_summaries (
    id TEXT PRIMARY KEY,
    review_summary TEXT,
    date DATETIME DEFAULT CURRENT_TIMESTAMP
)''')

In [4]:
conn.commit()

In [5]:
nltk.download('punkt')

def insert_newlines(text):
    sentences = nltk.sent_tokenize(text)
    text_with_newlines = '\n'.join(sentences)
    return text_with_newlines

[nltk_data] Downloading package punkt to /Users/flok/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Set api keys from env

In [6]:
#get api keys from env file
google_api_key = os.environ.get("GOOGLE_API_KEY")
openai_api_key = os.environ.get("OPENAI_API_KEY")
if google_api_key is not None:
    print("Google key found!")
if openai_api_key is not None:
    print("OpenAI key found!")

Google key found!
OpenAI key found!


### Define functions

In [11]:
#get nearby restaurants
def get_nearby_restaurants(api_key, location, radius=200, keyword='restaurant', num_results=5):
    '''
    Returns the restaurants from the API call based on location, radius
    '''
    # base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'
    
    params = {
        'location': location,
        'radius': radius,
        'keyword': keyword,
        'query': 'german food',
        'key': api_key,
    }
    
    response = requests.get(base_url, params=params)
    results = response.json().get('results', [])

    # Limit the number of results to the top 'num_results'
    results = results[:num_results]
    
    return results


def get_place_details(api_key, place_id):
    '''
    Returns details on a place defined by the place_id, e.g. reviews
    '''
    base_url = 'https://maps.googleapis.com/maps/api/place/details/json'

    params = {
        'place_id': place_id,
        'key': api_key
    }

    response = requests.get(base_url, params=params)
    result = response.json().get('result', {})
    
    photo_ref = result["photos"][0]["photo_reference"]
    
    response = requests.get(
        "https://maps.googleapis.com/maps/api/place/photo", 
        params={"photo_reference": photo_ref, "key": api_key, "maxwidth": 400}
    )
    photo_url = response.url
    
    address = result["formatted_address"]
    
    lat = result["geometry"]["location"]["lat"]
    lng = result["geometry"]["location"]["lng"]
    nav_address = address.replace(" ", "%20")
    
    return {
        "name": result["name"],
        "address": address,
        "photo_url": photo_url,
        "rating": result["rating"],
        "reviews": result["reviews"],
        "navigation_url": f"https://www.google.de/maps/dir/{lat},{lng}/{nav_address}"
    }


def generate_summary(client, reviews):
    #Concatenate all reviews
    review_text = "\n".join(review.get('text', '') for review in reviews)

    #Define the prompt for GPT
    prompt = f"Summarize the reviews for this place in one sentence:\n{review_text}"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    

    #Extract the generated summary from GPT's response
    summary = response.choices[0].message.content

    return summary

def fetch_review(place_id):
    cursor.execute('SELECT * FROM review_summaries WHERE id = ?', (place_id,))
    return cursor.fetchall()

def get_summary(place_id):
    review_array = fetch_review(place_id)

    if len(review_array) > 0:
        print("Found in database!")
        review = review_array[0]
        summary = review[1]
    else:
        reviews = get_place_details(google_api_key, place_id).get('reviews', [])
        summary = generate_summary(OpenAI(), reviews)
        cursor.execute('INSERT INTO review_summaries (id, review_summary) VALUES (?, ?)', (place_id, summary))
        conn.commit()

    return summary

def get_all_summaries(restaurants):
    summaries = []
    for restaurant in restaurants:
        summaries.append(get_summary(restaurant['place_id']))

    return summaries

### Search for places using Google Places API

In [12]:
# LMU coordiantes
# lmu_coordinates = '48.148794,11.579980'
lmu_coordinates = '48.13743, 11.57549'

# Set radius and keyword
restaurants = get_nearby_restaurants(google_api_key, lmu_coordinates, radius=500, keyword='food', num_results=5)

# Print the names of the found restaurants
for restaurant in restaurants:
    #Print Restaurant name and average rating
    print("Restaurant name: ", restaurant.get('name'))
    print("Rating:  ", restaurant.get('rating'))
    
    print(restaurant)

Restaurant name:  Nuernberger Bratwurst Gloeckl am Dom
Rating:   4.3
{'business_status': 'OPERATIONAL', 'formatted_address': 'Frauenplatz 9, 80331 München, Germany', 'geometry': {'location': {'lat': 48.1381846, 'lng': 11.5741547}, 'viewport': {'northeast': {'lat': 48.13996802989272, 'lng': 11.57524972989272}, 'southwest': {'lat': 48.13726837010727, 'lng': 11.57255007010728}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'icon_background_color': '#FF9E67', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet', 'name': 'Nuernberger Bratwurst Gloeckl am Dom', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1366, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102303653129799008016">Morawek &amp; Mencner Gastronomie GmbH</a>'], 'photo_reference': 'AWU5eFh43yfJ4qhbhYnQxq_yfJpkGy2phA8demc_wALhuVI85zHVazixoLxAa6iTje-H6ryc0QGh3Km2PH5z6Ugdct_VWZabcB1iW83GT8QRWRJGKdIndszK-qyjgHcV

In [13]:
get_place_details(google_api_key, restaurants[0]['place_id'])

{'name': 'Nuernberger Bratwurst Gloeckl am Dom',
 'address': 'Frauenplatz 9, 80331 München, Germany',
 'photo_url': 'https://lh3.googleusercontent.com/places/ANXAkqGsUCBOqio5gBDK2itiu0pMLI6m0t1ckx05GrMVkpmaEL3-Ct43B85bgyNhFKKclzjt_InOglDOSDjS9zk3YAXcGkfhOCagbHc=s1600-w400',
 'rating': 4.3,
 'reviews': [{'author_name': 'Michelle Hillock',
   'author_url': 'https://www.google.com/maps/contrib/106013331437514346797/reviews',
   'language': 'en',
   'original_language': 'en',
   'profile_photo_url': 'https://lh3.googleusercontent.com/a/ACg8ocJTgs4uXxehDbUCANp-FG0m6DwBfB2ZqHet3JPVWeNW=s128-c0x00000000-cc-rp-mo-ba6',
   'rating': 5,
   'relative_time_description': '2 months ago',
   'text': "We came here around 8pm on Monday and were seated right away at a 4 pp patio table. The server speaks English, the menu had 3 different languages. Very tourist friendly location. There was live music for a while. Smoking allowed outside + patio seating. Heated tents! Love the atmosphere since you can see

In [14]:
get_all_summaries(restaurants)

Found in database!
Found in database!
Found in database!
Found in database!
Found in database!


['and noisy. The food was tasty and authentic, but the portion sizes were large and difficult to finish. The waitstaff was friendly and attentive, but there were some issues with order accuracy. Overall, a great option for traditional Bavarian food in a tourist-friendly location.',
 'The place is busy but has a quick seating process, the food is decent but the sausage is not recommended, the roasted chicken is enjoyable, the washroom is underground, the service is great and the apple strudel is delicious, reservations are recommended, pork knuckle is crispy and service is excellent, some tourists felt treated poorly and had a negative experience with the waiter.',
 'Customers rave about the schnitzel at this restaurant in Munich, praising its taste and large portion sizes, and would highly recommend it to others. The service is pleasant and the drinks selection is decent, but be prepared to wait in line and bring cash as it is cash only.',
 'This place has an amazing atmosphere and exc

In [49]:
res_id = restaurants[0]["place_id"]

date = fetch_review(res_id)[0][2]

# check if date is older than 1 day from today

# if yes, generate new review and update database

# if no, use review from database


In [50]:
for elem in cursor.execute('SELECT * FROM review_summaries'):
    print(elem)

('5678', 'This is a review summary.', '2023-12-17 10:09:02')
('ChIJhf73vOt1nkcRds9gRrskPyo', 'Overall, the reviews for this place are generally positive, with praise for the crispy roasted mushrooms, decent Thai curry and dal soup options, free tap water, and good prices, while some negative aspects mentioned include tasteless dal, unfriendly staff, lack of toilet, and reliance on indoor ordering. The food is described as pre-prepped with quick service, and the atmosphere is described as young and fresh. The glass noodle salad and Thai curry are highly recommended, and there is appreciation for the knowledge of gluten-free and vegan options.', '2023-12-17 10:21:34')
('ChIJKUxiUul1nkcRewPz_a48sNQ', 'to this restaurant again.', '2023-12-17 10:32:33')
('ChIJSwyPQkh3nkcRdeTaWTm24oY', 'Customers enjoyed the tasty and high-quality Asian food, strongly recommending the place, with one customer being pleasantly surprised by the freshness and taste of the delivery sushi, while another customer 